# Installation & Imports

In [ ]:
pip install datasets

In [ ]:
pip install stanza

In [ ]:
pip install accelerate

In [ ]:
# these will ask you to restart the session which you must do to update the package requirements for this program
pip install -U spacy
pip install allennlp allennlp_models

  Using cached spacy-3.7.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
  Using cached thinc-8.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (922 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.3.3
    Uninstalling spacy-3.3.3:
      Successfully uninstalled spacy-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.1 requires spacy<3.4,>=2.1.0, but you have spacy 3.7.4 which is incompatible.


In [ ]:
#pip install allennlp allennlp_models

  Using cached spacy-3.3.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.3 MB)
  Using cached thinc-8.0.17-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (659 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.3
    Uninstalling thinc-8.2.3:
      Successfully uninstalled thinc-8.2.3
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.4
    Uninstalling spacy-3.7.4:
      Successfully uninstalled spacy-3.7.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.3.3 which is incompatible.


In [ ]:
import numpy
import pandas
import torch
import transformers
import stanza
import spacy
from spacy.matcher import Matcher
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
from datasets import load_dataset
import warnings

In [ ]:
# AllenNLP predictor download
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/biaffine-dependency-parser-ptb-2020.04.06.tar.gz")

In [ ]:
# Stanza download
stanza.download('en')
nlp_stanza = stanza.Pipeline('en')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
# spaCy download
nlp_spacy = spacy.load("en_core_web_sm")

# Stanza Setup and Testing

In [ ]:
# stanza testing
doc_stanza_test = nlp_stanza("Barack Obama was born in Hawaii.")
doc_spacy_test = nlp_spacy("Barack Obama was born in Hawaii.")

In [ ]:
print("Stanza Test:")
print(doc_stanza_test.entities)
print("spaCy Test:")
print(doc_spacy_test.ents)

Stanza Test:
[{
  "text": "Barack Obama",
  "type": "PERSON",
  "start_char": 0,
  "end_char": 12
}, {
  "text": "Hawaii",
  "type": "GPE",
  "start_char": 25,
  "end_char": 31
}]
spaCy Test:
(Barack Obama, Hawaii)


# Load Spider Dataset

In [ ]:
dataset = load_dataset("spider", split='train')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/spider-d5d87732c0c2de44/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [ ]:
dataset

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 7000
})

In [ ]:
dataset['query']

['SELECT count(*) FROM head WHERE age  >  56',
 'SELECT name ,  born_state ,  age FROM head ORDER BY age',
 'SELECT creation ,  name ,  budget_in_billions FROM department',
 'SELECT max(budget_in_billions) ,  min(budget_in_billions) FROM department',
 'SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15',
 "SELECT name FROM head WHERE born_state != 'California'",
 "SELECT DISTINCT T1.creation FROM department AS T1 JOIN management AS T2 ON T1.department_id  =  T2.department_id JOIN head AS T3 ON T2.head_id  =  T3.head_id WHERE T3.born_state  =  'Alabama'",
 'SELECT born_state FROM head GROUP BY born_state HAVING count(*)  >=  3',
 'SELECT creation FROM department GROUP BY creation ORDER BY count(*) DESC LIMIT 1',
 "SELECT T1.name ,  T1.num_employees FROM department AS T1 JOIN management AS T2 ON T1.department_id  =  T2.department_id WHERE T2.temporary_acting  =  'Yes'",
 'SELECT count(DISTINCT temporary_acting) FROM management',
 'SELECT count(*) FROM department WH

# Parse Dataset with Stanza and Save to File

In [ ]:
def parse_with_stanford(parser, dataset):
    results = []
    for example in dataset:
        parsed_output = parser(example['question'])
        results.append({
            'input': example['question'],
            'predicted': parsed_output,
            'gold_standard': example['query']
        })
    return results

In [ ]:
x = parse_with_stanford(nlp_stanza, dataset)

In [ ]:
import json

file_path = 'parsed_results.json'

with open(file_path, 'w') as f:
    json.dump(x, f)

print("Results have been saved to", file_path)

In [ ]:
print(x[:5])

[{'input': 'How many heads of the departments are older than 56 ?', 'predicted': [
  [
    {
      "id": 1,
      "text": "How",
      "lemma": "how",
      "upos": "ADV",
      "xpos": "WRB",
      "feats": "PronType=Int",
      "head": 2,
      "deprel": "advmod",
      "start_char": 0,
      "end_char": 3,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 2,
      "text": "many",
      "lemma": "many",
      "upos": "ADJ",
      "xpos": "JJ",
      "feats": "Degree=Pos",
      "head": 3,
      "deprel": "amod",
      "start_char": 4,
      "end_char": 8,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 3,
      "text": "heads",
      "lemma": "head",
      "upos": "NOUN",
      "xpos": "NNS",
      "feats": "Number=Plur",
      "head": 8,
      "deprel": "nsubj",
      "start_char": 9,
      "end_char": 14,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "of",
  

In [ ]:
with open('file.txt', 'w') as f:
    data = []
    for line in x:
        f.write(line + '\n')
        print(line)

In [ ]:
import pickle
with open("parsePickle", "wb") as fp:
  pickle.dump(x, fp)

In [ ]:
with open("parsePickle", "rb") as fp:
   b = pickle.load(fp)

# Parsing

In [ ]:
# dependency parsing which shows the relationship between words in the sentence
def dependency_parsing(doc):
  for sentence in doc.sentences:
      for word in sentence.words:
          print(f"word: {word.text}, parent: {sentence.words[word.head-1].text if word.head > 0 else 'root'}, relation: {word.deprel}")

In [ ]:
def stanza_nlp_to_sql(doc):
    select_clause = []
    from_clause = []
    where_clause = []
    conditions = []

    # Identify SQL Components (parts of speech like noun, verb, preposition)
    for sentence in doc.sentences:
        for word in sentence.words:
            # Rule for identifying possible SELECT attributes
            if word.upos == "NOUN" and word.xpos in ["NN", "NNS"]:
                if word.text.lower() not in ["number", "count", "total"]:  # avoiding aggregate functions for simplicity
                    select_clause.append(word.text)

            # Detecting possible FROM clause values
            if word.deprel == "nmod" and word.upos == "PROPN":
                from_clause.append(word.text)

            # Building WHERE clause based on adjectives or numeric modifiers
            if word.deprel in ["amod", "nummod"] and word.head > 0:
                head_word = sentence.words[word.head - 1].text
                conditions.append(f"{head_word} = '{word.text}'")

    # Constructing clauses from identified parts
    if not select_clause:  # default select clause if none identified
        select_clause.append("*")
    select_part = "SELECT " + ", ".join(select_clause)
    from_part = "FROM " + ", ".join(from_clause) if from_clause else "FROM [TABLE]"  # Placeholder if no table identified
    where_part = "WHERE " + " AND ".join(conditions) if conditions else ""

    # Combining parts to form final SQL query
    sql_query = f"{select_part} {from_part} {where_part}"
    return sql_query.strip()


# # Stanza test
# doc = nlp("How many heads of the departments are older than 56 ?")
# sql_query = stanza_nlp_to_sql(doc)
# print(sql_query)

In [ ]:
def spacy_nlp_to_sql(doc):
    select_clause = []
    from_clause = []
    where_clause = []
    conditions = []

    for token in doc:
        if token.dep_ == 'attr' or token.dep_ == 'dobj':
            # Attribute directly follows "What is/are"
            select_clause = token.text
        elif token.dep_ == 'prep' and token.text.lower() == 'of':
            # Entity follows the preposition "of"
            if token.head.dep_ in ('attr', 'dobj') and token.nbor().pos_ != 'DET':
                from_clause = token.nbor().text
        elif token.dep_ == 'nsubj':
            if token.head.text.lower() in ('is', 'are'):
                select_clause = token.text

    # Search for a simple WHERE clause
    for token in doc:
        if token.dep_ == 'prep' and token.text.lower() == 'where':
            condition = [token.nbor().text, token.nbor().nbor().text, token.nbor().nbor().nbor().text]
            where_clause = " ".join(condition)

    # Formulate SQL
    sql = f"SELECT {select_clause} FROM {from_clause}"
    if where_clause:
        sql += f" WHERE {where_clause}"

    return sql

# # spaCy test
# nlp = spacy.load("en_core_web_sm")
# doc = nlp("How many heads of the departments are older than 56 ?")
# sql_query = spacy_nlp_to_sql(doc)
# print(sql_query)

In [ ]:
def allen_nlp_to_sql(doc):
    select_clause = []
    from_clause = []
    where_clause = []
    conditions = []

    # Process each token in the parsed document
    for token in doc['words']:
        idx = doc['words'].index(token)
        pos_tag = doc['pos'][idx]
        dep_rel = doc['predicted_dependencies'][idx]
        head_idx = doc['predicted_heads'][idx] - 1  # Adjusting because AllenNLP uses 1-based indexing
        head_word = doc['words'][head_idx] if head_idx >= 0 else None

        # Rule for identifying possible SELECT attributes
        if pos_tag.startswith("NN"):  # Simplified POS check
            if token.lower() not in ["number", "count", "total"]:  # avoiding aggregate functions for simplicity
                select_clause.append(token)

        # Detecting possible FROM clause values
        if dep_rel == "nmod" and pos_tag.startswith("NNP"):
            from_clause.append(token)

        # Building WHERE clause based on adjectives or numeric modifiers
        if dep_rel in ["amod", "nummod"] and head_word:
            conditions.append(f"{head_word} = '{token}'")

    # Constructing clauses from identified parts
    if not select_clause:  # default select clause if none identified
        select_clause.append("*")
    select_part = "SELECT " + ", ".join(select_clause)
    from_part = "FROM " + ", ".join(from_clause) if from_clause else "FROM [TABLE]"  # Placeholder if no table identified
    where_part = "WHERE " + " AND ".join(conditions) if conditions else ""

    # Combine parts to form final SQL query
    sql_query = f"{select_part} {from_part} {where_part}"
    return sql_query.strip()

# # Allen test
# #predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/biaffine-dependency-parser-ptb-2020.04.06.tar.gz")
# sentence = "How many heads of the departments are older than 56 ?"
# parsed_output = predictor.predict(sentence=sentence)

# sql_query = allen_nlp_to_sql(parsed_output)
# print(sql_query)


In [ ]:
def weak_supervision(sql_candidates, query):
    def score_sql(sql):
        score = 0
        # Heuristic 1: Reward SQL queries that use keywords likely to be in the correct query
        important_keywords = ['SELECT', 'FROM', 'WHERE', 'JOIN', 'ORDER BY', 'GROUP BY']
        for keyword in important_keywords:
            if keyword in sql:
                score += 1

        # Heuristic 2: Penalize overly complex queries (simplicity bias)
        if sql.count(',') > 3 or len(sql.split()) > 10:
            score -= 1

        # Heuristic 3: Check if query terms appear in the SQL (simple context matching)
        query_terms = query.lower().split()
        sql_terms = sql.lower().split()
        common_terms = set(query_terms).intersection(sql_terms)
        score += len(common_terms)

        return score

    # Score each SQL candidate and select the one with the highest score
    best_sql = max(sql_candidates, key=score_sql)
    return best_sql


In [ ]:
def basic_confidence_score(query, sql_query):
    # Calculates a basic confidence score based on simple heuristics.


    doc = nlp(query)

    # Heuristic 1: Length of the query (longer queries might be more complex)
    query_length_score = len(doc) / 20.0  # assuming a max length of 20 for normalization

    # Heuristic 2: Presence of specific keywords in SQL (SELECT, WHERE, FROM)
    essential_keywords = ['SELECT', 'WHERE', 'FROM']
    keyword_score = sum(1 for keyword in essential_keywords if keyword in sql_query.upper())
    keyword_score /= len(essential_keywords)

    # Heuristic 3: Basic match score between query entities and SQL columns/tables
    query_tokens = set(token.text.lower() for token in doc if not token.is_stop and token.is_alpha)
    sql_tokens = set(sql_query.lower().split())
    match_score = len(query_tokens & sql_tokens) / len(query_tokens.union(sql_tokens))

    # Aggregate the scores to produce a confidence score
    confidence_score = (query_length_score + keyword_score + match_score) / 3.0
    return confidence_score

def uncertainty_quantification(query, sql_query, threshold=0.5):
    # We quantify the uncertainty of the SQL query and decide whether to ask for clarification.

    confidence_score = basic_confidence_score(query, sql_query)
    if confidence_score < threshold:
        return False, f"Confidence is low ({confidence_score:.2f}). Please provide more information or rephrase your query."
    return True, f"Query is confident ({confidence_score:.2f})."

# # testing
# query = "List all employees who are older than 30."
# sql_query = "SELECT * FROM employees WHERE age > 30"

# is_confident, message = uncertainty_quantification(query, sql_query)
# print(message)

In [ ]:
# Limited to 15 queries to save space since there are ten thousand possible entries
max_entries = 15
entries_processed = 0

# Loop to run through entries from dataset
for entry in dataset:

    if entries_processed >= max_entries:
          break

    # Grab the correct english sentence and the corresponsing correct SQL query
    sentence = entry["question"]
    correct_query = entry["query"]

    # Dependency Parsing
    print("Dependency Parse:")
    doc_dependency = nlp_stanza(sentence)
    dependency_parse = dependency_parsing(doc_dependency)
    print(dependency_parse)
    print("\n")

    # Stanza to SQL
    doc_stanza = nlp_stanza(sentence)
    sql_query_stanza = stanza_nlp_to_sql(doc_stanza)
    print("Stanza Parse-to-SQL:")
    print(sql_query_stanza)
    print()

    # spaCy to SQL
    doc_spacy = nlp_spacy(sentence)
    sql_query_spacy = spacy_nlp_to_sql(doc_spacy)
    print("spaCy Parse-to-SQL:")
    print(sql_query_spacy)
    print()

    # AllenNLP to SQL
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        doc_Allen = predictor.predict(sentence=sentence)
        sql_query_Allen = allen_nlp_to_sql(doc_Allen)
        print("AllenNLP Parse-to-SQL:")
        print(sql_query_Allen)
        print()

    # Store SQL queries into a list for weak supervision
    sql_queries = [sql_query_stanza, sql_query_spacy, sql_query_Allen]

    # Weak Supervision
    weak_supervision_result = weak_supervision(sql_queries, correct_query)
    print("Best Query Result after Weak Supervision:")
    print(weak_supervision_result)
    print()

    # Uncertainty Quantification
    query = sentence
    sql_query = weak_supervision_result

    is_confident, message = uncertainty_quantification(query, sql_query)
    print(message)

    entries_processed += 1

Dependency Parse:
word: How, parent: many, relation: advmod
word: many, parent: heads, relation: amod
word: heads, parent: older, relation: nsubj
word: of, parent: departments, relation: case
word: the, parent: departments, relation: det
word: departments, parent: heads, relation: nmod
word: are, parent: older, relation: cop
word: older, parent: root, relation: root
word: than, parent: 56, relation: case
word: 56, parent: older, relation: obl
word: ?, parent: older, relation: punct
None


Stanza Parse-to-SQL:
SELECT heads, departments FROM [TABLE] WHERE heads = 'many'

spaCy Parse-to-SQL:
SELECT heads FROM []

AllenNLP Parse-to-SQL:
SELECT * FROM [TABLE] WHERE heads = 'older'

Best Query Result after Weak Supervision:
SELECT heads, departments FROM [TABLE] WHERE heads = 'many'

Query is confident (0.58).
Dependency Parse:
word: List, parent: root, relation: root
word: the, parent: name, relation: det
word: name, parent: List, relation: obj
word: ,, parent: state, relation: punct
word: 

# Overfitted Spacy Function (We decided not to use this in our project since it is too specific and overfits to certain sentence structures while underperforming for other sentence structures.

In [ ]:
nlp = spacy.load('en_core_web_sm')

def extract_conditions(doc):
    # Extracting conditions using simple pattern matching
    matcher = Matcher(nlp.vocab)
    pattern = [{'LOWER': 'older', 'OP': '?'}, {'LIKE_NUM': True}]
    matcher.add("NUMERIC_CONDITION", [pattern])
    matches = matcher(doc)
    conditions = []
    for match_id, start, end in matches:
        span = doc[start:end]
        if span.text.lower().startswith('older'):
            condition = f"age > {span.text.split()[-1]}"
        else:
            condition = f"age = {span.text}"
        conditions.append(condition)
    return " AND ".join(conditions)

def nl_to_sql_universal(query):
    doc = nlp(query)

    select_clause = "SELECT COUNT(*)"
    from_clause = "FROM employees"
    where_clause = ""

    # Extract conditions
    where_clause = extract_conditions(doc)

    # Construct SQL query
    if where_clause:
        sql_query = f"{select_clause} {from_clause} WHERE {where_clause}"
    else:
        sql_query = f"{select_clause} {from_clause}"

    return sql_query

query = "How many heads of the departments are older than 56 ??"
sql_query = nl_to_sql_universal(query)
print(sql_query)

SELECT COUNT(*) FROM employees WHERE age = 56
